In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets 
from torchvision import transforms
from torchvision.utils import save_image
from torch.autograd import Variable


import pandas as pd 
import numpy as np
import os 
import glob
np.random.seed(1337)  # for reproducibility
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import torch.optim as optim
import torch
import torch.nn.functional as nn
import torch.autograd as autograd
import torch.optim as optim
import numpy as np
#import matplotlib.pyplot as plt
#import matplotlib.gridspec as gridspec
import os
from torch.autograd import Variable
from sklearn.utils import shuffle
import random
from keras.datasets import mnist
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras import optimizers

Using TensorFlow backend.


In [2]:
train_lre_10sec = pd.read_csv('/home/satishk/GAN_lre/gan_csv/GAN_10sec_ivectors_X_train_04Jan_labels_ids_combined_extraction.csv')
#train_afds

In [3]:
train_lre_10sec.head()

,0,1,2,3,4,5,6,7,8,9,...,495,496,497,498,499,ids,year,data,lang,lang_id
0,-0.314181,0.671920,-0.188068,-0.910752,1.050361,1.966829,0.292740,1.084547,1.287480,1.745248,...,-0.564911,-1.854116,-3.185948,-2.177057,0.301166,zkllk,LDC2017E22,data,eng-usg,zkllk
1,0.503734,0.247072,0.451463,0.500546,-0.716668,1.337575,-1.027019,0.773720,0.910362,1.728001,...,0.886613,6.766799,-1.040380,-0.751316,-1.487938,lid05e1_lid00562,LDC2017E22,data,eng-usg,lid05e1_lid00562
2,2.141677,-0.044947,-0.595182,-0.595953,0.889385,1.440080,-1.208933,0.129059,-0.930121,0.401362,...,-2.836253,-0.323334,0.055760,0.481039,0.395826,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a
3,1.708722,-0.663542,0.031733,0.358099,0.594610,1.023025,-1.450633,0.252159,-0.381832,0.087693,...,1.384728,0.426863,0.482178,1.655650,-1.029032,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a
4,1.873292,-0.272493,-0.305948,-0.250047,2.093700,1.741634,-0.750633,-0.651771,-0.793287,-0.053527,...,0.188493,-0.888710,1.792536,-0.628455,1.912622,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a


In [4]:
X_train_10sec = train_lre_10sec.drop(['ids','year','data','lang','lang_id'],axis=1)

In [5]:
#train_lre = train_lre.iloc[1000:2000]

In [6]:
train_lre_30sec = pd.read_csv('/home/satishk/GAN_lre/gan_csv/GAN_30sec_ivectors_X_train_04Jan_labels_ids_combined_extraction.csv')


In [7]:
train_lre_30sec.head()

,0,1,2,3,4,5,6,7,8,9,...,495,496,497,498,499,ids,year,data,lang,lang_id
0,-0.639420,0.345684,-0.517645,-0.737002,1.313001,1.655732,0.615168,0.799339,1.648419,1.314986,...,-1.322407,-0.717348,-3.843951,-1.471274,0.945421,zkllk,LDC2017E22,data,eng-usg,zkllk
1,0.113610,0.738034,0.584856,-0.248521,0.256616,1.060159,-0.416211,0.133670,0.188327,0.805241,...,1.965741,3.525555,0.006925,1.676099,-1.116371,lid05e1_lid00562,LDC2017E22,data,eng-usg,lid05e1_lid00562
2,2.061305,-0.140605,-0.627104,-0.682908,1.337613,1.261543,-0.651291,0.307040,-0.980717,0.335530,...,-3.018432,-1.841361,0.814595,-0.200731,1.180795,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a
3,1.958049,-0.608315,-0.113532,0.167272,0.911100,1.116094,-0.847214,-0.041357,-0.542335,0.199105,...,-0.548767,-1.254447,1.644647,0.555434,1.159140,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a
4,1.842436,-0.417598,-0.427428,-0.485898,1.204330,1.439188,-1.056642,0.111603,-0.404313,-0.307075,...,0.297026,-1.858708,0.209784,0.583939,1.398789,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a


In [8]:
X_train_30sec = train_lre_30sec.drop(['ids','year','data','lang','lang_id'],axis=1)

In [9]:
y_30sec_labels = train_lre_30sec["lang"]

In [10]:
y_30sec_labels.head()

0    eng-usg
1    eng-usg
2    ara-apc
3    ara-apc
4    ara-apc
Name: lang, dtype: object

In [11]:
le = preprocessing.LabelEncoder()
le.fit(y_30sec_labels)

LabelEncoder()

In [12]:
le.classes_

array(['ara-acm', 'ara-apc', 'ara-ary', 'ara-arz', 'eng-gbr', 'eng-usg',
       'por-brz', 'qsl-pol', 'qsl-rus', 'spa-car', 'spa-eur', 'spa-lac',
       'zho-cmn', 'zho-nan'], dtype=object)

In [13]:
y_30sec_labels=le.transform(y_30sec_labels)

In [14]:
y_30sec_labels.shape

(114276,)

In [15]:
nb_classes = 14
# convert class vectors to binary class matrices
Y_train_labels = np_utils.to_categorical(y_30sec_labels, nb_classes)

In [16]:
Y_train_labels.shape

(114276, 14)

In [17]:
X_train_10sec = X_train_10sec.values
X_train_30sec = X_train_30sec.values

In [18]:
X_train_10sec = X_train_10sec.astype('float32')
#X_test = X_test.astype('float32')
X_train_30sec = X_train_30sec.astype('float32')

#X_train /= 255
#X_test /= 255
#X_val /= 255

print('X_train_10sec shape:', X_train_10sec.shape)
print(X_train_10sec.shape[0], 'train 10sec')
#print(X_test.shape[0], 'test samples')
print(X_train_30sec.shape[0], ' train 30sec')


X_train_10sec shape: (114276, 500)
114276 train 10sec
114276  train 30sec


In [19]:
#Shuffle the Dataset
X_train_10sec_F,  X_train_30sec_F, Y_train_labels = shuffle(X_train_10sec, X_train_30sec, Y_train_labels, random_state=0)

In [20]:
import torch.nn as nn

In [21]:
class FC_Network(torch.nn.Module):

    def __init__(self,):
        super(FC_Network, self).__init__()

        D_in =  114
        D_out =500      
        num_nodes_fnn = 2048
        self.layer_1 = torch.nn.Linear(D_in, num_nodes_fnn)
        self.bn_1 = torch.nn.BatchNorm1d(num_nodes_fnn)
        self.layer_2 = torch.nn.Linear(num_nodes_fnn, num_nodes_fnn)
        self.bn_2 = torch.nn.BatchNorm1d(num_nodes_fnn)
        self.layer_3 = torch.nn.Linear(num_nodes_fnn, num_nodes_fnn)
        self.bn_3 = torch.nn.BatchNorm1d(num_nodes_fnn)
        self.layer_4 = torch.nn.Linear(num_nodes_fnn,num_nodes_fnn)
        self.bn_4 = torch.nn.BatchNorm1d(num_nodes_fnn)
        self.output_layer = torch.nn.Linear(num_nodes_fnn, D_out)
        self.Lrelu = torch.nn.LeakyReLU()
        self.dropout = torch.nn.Dropout(p=0.5)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x, labels):
        out = torch.cat([x, labels], 1)
        #out = x.view(x.size(0), -1)
        out = self.dropout((self.Lrelu(self.bn_1(self.layer_1(out)))))
        out = self.dropout((self.Lrelu(self.bn_2(self.layer_2(out)))))
        out = self.dropout((self.Lrelu(self.bn_3(self.layer_3(out)))))
        out = self.dropout((self.Lrelu(self.bn_4(self.layer_4(out)))))
        out = self.sigmoid(self.output_layer(out))
        #out = (self.relu(self.layer_1(out)))
        #out = (self.relu(self.layer_2(out)))
        #out = (self.relu(self.layer_3(out)))
        #out = (self.relu(self.layer_4(out)))
        #out = (self.output_layer(out))
        #out = out.view(x.size())
        return out

In [22]:
S_encoder = FC_Network()

In [23]:
S_encoder.cuda()

FC_Network(
  (layer_1): Linear(in_features=114, out_features=2048)
  (bn_1): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True)
  (layer_2): Linear(in_features=2048, out_features=2048)
  (bn_2): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True)
  (layer_3): Linear(in_features=2048, out_features=2048)
  (bn_3): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True)
  (layer_4): Linear(in_features=2048, out_features=2048)
  (bn_4): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True)
  (output_layer): Linear(in_features=2048, out_features=500)
  (Lrelu): LeakyReLU(0.01)
  (dropout): Dropout(p=0.5)
  (sigmoid): Sigmoid()
)

In [24]:
lr = 1e-4
#betas = (0.5, 0.999)

In [25]:
# Loss function
#criterion = torch.nn.MSELoss()

# Optimizers
S_solver = optim.RMSprop(S_encoder.parameters(), lr=lr)
#S_solver = torch.optim.Adam(S_encoder.parameters(), lr=learning_rate, betas=betas)
#T_solver = torch.optim.Adam(T_encoder.parameters(), lr=learning_rate/2, betas=betas)

In [26]:
len(X_train_10sec)

114276

In [27]:

len(X_train_30sec)

114276

In [28]:
mb_size = 256
batch_size = mb_size

In [29]:
#Implementation of Discriminator

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        D_in =  514
        D_out = 1     
        num_nodes_fnn = 512
        self.layer_1 = torch.nn.Linear(D_in, num_nodes_fnn)
        self.bn_1 = torch.nn.BatchNorm1d(num_nodes_fnn)
        self.layer_2 = torch.nn.Linear(num_nodes_fnn, num_nodes_fnn)
        self.bn_2 = torch.nn.BatchNorm1d(num_nodes_fnn)
        self.layer_3 = torch.nn.Linear(num_nodes_fnn, num_nodes_fnn)
        self.bn_3 = torch.nn.BatchNorm1d(num_nodes_fnn)
        self.output_layer = torch.nn.Linear(num_nodes_fnn, D_out)
        self.Lrelu = torch.nn.LeakyReLU()
        #self.sigmoid = torch.nn.Sigmoid()
        
    def forward(self, x, labels ):
        out = torch.cat([x, labels], 1)
        out = self.Lrelu(self.bn_1(self.layer_1(out)))
        out = self.Lrelu(self.bn_2(self.layer_2(out)))
        out = self.Lrelu(self.bn_3(self.layer_3(out)))
        out = (self.output_layer(out))
        return out

In [30]:
D = Discriminator()

In [31]:
D.cuda()

Discriminator(
  (layer_1): Linear(in_features=514, out_features=512)
  (bn_1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True)
  (layer_2): Linear(in_features=512, out_features=512)
  (bn_2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True)
  (layer_3): Linear(in_features=512, out_features=512)
  (bn_3): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True)
  (output_layer): Linear(in_features=512, out_features=1)
  (Lrelu): LeakyReLU(0.01)
)

In [32]:
S_encoder

FC_Network(
  (layer_1): Linear(in_features=114, out_features=2048)
  (bn_1): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True)
  (layer_2): Linear(in_features=2048, out_features=2048)
  (bn_2): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True)
  (layer_3): Linear(in_features=2048, out_features=2048)
  (bn_3): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True)
  (layer_4): Linear(in_features=2048, out_features=2048)
  (bn_4): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True)
  (output_layer): Linear(in_features=2048, out_features=500)
  (Lrelu): LeakyReLU(0.01)
  (dropout): Dropout(p=0.5)
  (sigmoid): Sigmoid()
)

In [33]:
ones_label = Variable(torch.ones(mb_size, 1))
zeros_label = Variable(torch.zeros(mb_size,1))
#ones_label_fake = Variable(torch.ones(mb_size*2))

In [34]:
ones_label = ones_label.cuda()
zeros_label = zeros_label.cuda()

In [35]:
# Optimizers
#T_solver = torch.optim.Adam(T_encoder.parameters(), lr=learning_rate, betas=betas)
D_solver = optim.RMSprop(D.parameters(), lr=lr)
#D_solver = torch.optim.SGD(D.parameters(), lr = 0.001)

In [36]:
from torch.nn.functional import binary_cross_entropy

In [37]:
D_iter = 5
mb_size = 256
Z_dim = 100
batch_size = mb_size
k = 0
# Start training
for epoch in range(200):
    
    

    print('epoch:',epoch)
    #for i in range(XX_train):
    # Build mini-batch dataset
    #batch_size = images.size(0)
    #images = to_var(images.view(batch_size, -1))

    it=0
    j = 0
    while it+batch_size < len(X_train_10sec_F) :
        
        
        start= it
        end= it + batch_size
        for p in D.parameters(): # reset requires_grad
                p.requires_grad = True # they are set to False below in netG update

        
        while j < D_iter and it < len(X_train_10sec_F)-batch_size:
            
            
            # Weight clipping
            for p in D.parameters():
                p.data.clamp_(-0.001, 0.001)
                
            # Housekeeping - reset gradient
            D.zero_grad()    
            #start= it
            #end= it + batch_size
            #for p in D.parameters(): # reset requires_grad
                #p.requires_grad = True # they are set to False below in netG update
            z = np.random.randn(mb_size, Z_dim)
            z = Variable(torch.from_numpy(z.astype('float32')))
            r = np.random.randint(X_train_30sec_F.shape[0], size=256)
            X_D = X_train_10sec_F[r, :]
            #c_D = X_train_30sec_F[r, :]
            
            #X = X_train_10sec_F[start:end]

            labels_D = Y_train_labels[r, :]

            #c = X_train_30sec_F[start:end]
            X = Variable(torch.from_numpy(X_D))
            #c = Variable(torch.from_numpy(c_D.astype('float32')))
            labels = Variable(torch.from_numpy(labels_D.astype('float32')))
            X = X.cuda()
            z = z.cuda()
            #c = c.cuda()
            labels = labels.cuda()
            # Dicriminator forward-loss-backward-update
            S_sample = S_encoder(z, labels)
            #T_sample = T_encoder(c)
            D_real = D(X, labels)
            D_fake = D(S_sample, labels)

            D_loss = -(torch.mean(D_real) - torch.mean(D_fake))

            D_loss.backward()
            D_solver.step()
            
            j += 1
            #print('j=',j)
            
            
        # Generator forward-loss-backward-update
        #start= it
        #end= it + batch_size

        X = X_train_10sec_F[start:end]
        labels = Y_train_labels[start:end]

        #c = X_train_30sec_F[start:end]
        X = Variable(torch.from_numpy(X))
        #c = Variable(torch.from_numpy(c.astype('float32')))
        labels = Variable(torch.from_numpy(labels.astype('float32')))
        X = X.cuda()
        #c = c.cuda()
        labels = labels.cuda()
        #z = Variable(torch.randn(mb_size, Z_dim))
        #G_sample = G(z, c)
        
        # Housekeeping - reset gradient
        D.zero_grad()
        S_encoder.zero_grad()

        for p in D.parameters():
            p.requires_grad = False # to avoid computation
            
        S_sample = S_encoder(z, labels)
        D_fake = D(S_sample, labels)
        #D_fake = D(G_sample, c)
        #T_loss = criterion(T_sample, c)
        S_loss = -torch.mean(D_fake)
        #S_loss = binary_cross_entropy(D_fake, ones_label)

        S_loss.backward()
        S_solver.step()
        k = k+1
        #print('k=',k)
        #S_encoder.zero_grad()
        # Housekeeping - reset gradient
        #D.zero_grad()

        
        j = 0
        it+= batch_size
    print('S_loss: {}'.format(S_loss.cpu().data.numpy()),'D_loss: {}'.format(D_loss.cpu().data.numpy()))
           

epoch: 0
S_loss: [ 0.00098358] D_loss: [-0.]
epoch: 1
S_loss: [ 0.00098447] D_loss: [-0.]
epoch: 2
S_loss: [ 0.00098398] D_loss: [-0.]
epoch: 3
S_loss: [ 0.00098471] D_loss: [-0.]
epoch: 4
S_loss: [ 0.00098607] D_loss: [-0.]
epoch: 5


KeyboardInterrupt: 

In [74]:
S_encoder.eval()

FC_Network(
  (layer_1): Linear(in_features=114, out_features=2048)
  (bn_1): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True)
  (layer_2): Linear(in_features=2048, out_features=2048)
  (bn_2): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True)
  (layer_3): Linear(in_features=2048, out_features=2048)
  (bn_3): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True)
  (layer_4): Linear(in_features=2048, out_features=2048)
  (bn_4): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True)
  (output_layer): Linear(in_features=2048, out_features=500)
  (Lrelu): LeakyReLU(0.01)
  (dropout): Dropout(p=0.5)
  (sigmoid): Sigmoid()
)

In [75]:
df_SF=pd.DataFrame()
y_dim = 14
samples_per_class = 5000
#c = np.zeros(shape=[samples_per_class, y_dim], dtype='float32')
#c[:, np.random.randint(0, 10)] = 1.
for i in range(14):
    #print(i)
    c = np.zeros(shape=[samples_per_class, y_dim], dtype='float32')
    c[:, i] = 1.
    c_df=pd.DataFrame(c)
    df_SF = df_SF.append(c_df,ignore_index = True)
    

In [76]:
c_df.shape

(5000, 14)

In [77]:
df_SF.shape[0]

70000

In [78]:
c_gen = df_SF.values

In [79]:
z_gen = Variable(torch.randn(df_SF.shape[0], Z_dim))

In [80]:
z_gen.data.shape

torch.Size([70000, 100])

In [81]:
c_gen = Variable(torch.from_numpy(c_gen))

In [82]:
S_encoder.cpu()

FC_Network(
  (layer_1): Linear(in_features=114, out_features=2048)
  (bn_1): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True)
  (layer_2): Linear(in_features=2048, out_features=2048)
  (bn_2): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True)
  (layer_3): Linear(in_features=2048, out_features=2048)
  (bn_3): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True)
  (layer_4): Linear(in_features=2048, out_features=2048)
  (bn_4): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True)
  (output_layer): Linear(in_features=2048, out_features=500)
  (Lrelu): LeakyReLU(0.01)
  (dropout): Dropout(p=0.5)
  (sigmoid): Sigmoid()
)

In [83]:
samples = S_encoder(z_gen, c_gen)

In [84]:
samples.data.shape

torch.Size([70000, 500])

In [85]:
#Taking the generated iVectors we will try to check the acc by MLP

In [86]:
train_X1 = samples.cpu().data.numpy()

In [87]:
type(train_X1)

numpy.ndarray

In [88]:
train_y1 = c_gen.cpu().data.numpy()

In [89]:
type(train_y1)

numpy.ndarray

In [90]:
train_y1

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]], dtype=float32)

In [91]:
train_X1 = pd.DataFrame(train_X1)
train_y1 = pd.DataFrame(train_y1)

In [92]:
train_X1,  train_y1 = shuffle(train_X1, train_y1, random_state=0)

In [93]:
val_lre = pd.read_csv('/home/satishk/GAN_lre/gan_csv/dev_feat_BNF_h5_05Jan_10sec.csv')


In [94]:
val_lre.head()

,0,1,2,3,4,5,6,7,8,9,...,495,496,497,498,499,uttid,language_code,data_source,speech_duration,segmentid1
0,-0.195398,-0.390386,-0.624269,1.535433,0.388117,1.727448,0.463109,2.059767,0.165206,0.619942,...,0.616712,-2.654504,0.482690,0.401316,-1.639292,lre17_zpuysjrb,zho-nan,mls14,10,lre17_zpuysjrb
1,-0.629280,-0.063858,0.397267,-0.205915,0.102730,2.412635,1.007062,1.123458,2.397956,1.009312,...,-0.626052,1.390082,0.393644,1.559527,-1.124127,lre17_ljfobgxa,eng-usg,mls14,10,lre17_ljfobgxa
2,0.896452,0.746032,-0.547790,1.955862,-0.198707,1.430271,0.535072,1.199097,2.253311,0.050932,...,-1.859781,-0.889882,-1.823634,-1.507139,-0.811382,lre17_guhfzbxl,zho-cmn,mls14,10,lre17_guhfzbxl
3,0.323422,0.577406,1.585811,-0.071801,-0.398092,1.788636,1.529049,1.042687,-0.557043,1.131491,...,0.519515,1.404486,-0.052511,0.367571,1.274410,lre17_tbywroez,eng-usg,mls14,10,lre17_tbywroez
4,1.384084,0.814175,1.261965,1.078853,-1.601087,0.756707,1.157398,0.146018,-0.566307,-0.446297,...,-0.850699,0.355410,-1.455804,2.629600,-0.044268,lre17_phsyjibn,spa-lac,mls14,10,lre17_phsyjibn


In [95]:
X_val = val_lre.drop(["language_code","uttid","segmentid1","data_source","speech_duration"],axis=1)
y_val = val_lre["language_code"]
y_val_segmentid = val_lre["segmentid1"]

In [96]:
X_val = X_val.values
#X_eval = X_eval.values

In [97]:
le = preprocessing.LabelEncoder()
le.fit(y_val)

LabelEncoder()

In [98]:
le.classes_

array(['ara-acm', 'ara-apc', 'ara-ary', 'ara-arz', 'eng-gbr', 'eng-usg',
       'por-brz', 'qsl-pol', 'qsl-rus', 'spa-car', 'spa-eur', 'spa-lac',
       'zho-cmn', 'zho-nan'], dtype=object)

In [99]:
y_val_labels = le.transform(y_val)

In [100]:
y_val_labels[0:25]

array([13,  5, 12,  5, 11, 11, 11, 12, 10,  0, 11,  0,  2, 13, 12,  6, 13,
        4,  6,  0, 11,  2,  4,  3, 10])

In [101]:
# convert class vectors to binary class matrices
Y_val = np_utils.to_categorical(y_val_labels, nb_classes)
#Y_eval = np_utils.to_categorical(y_eval_labels, nb_classes)

In [102]:
Y_val

array([[ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.]])

In [103]:
X_val = X_val.astype('float32')

In [104]:
X_val.shape

(928, 500)

In [105]:
model = Sequential()
model.add(Dense(512, input_dim=500))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [106]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 512)               256512    
_________________________________________________________________
activation_4 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               262656    
_________________________________________________________________
activation_5 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 14)                7182      
__________

In [107]:
adam=optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [108]:
best_weights_filepath = '/home/satishk/saved_weights/best_weights_2l_MLP_11.hdf5'
saveBestModel = ModelCheckpoint(best_weights_filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='auto')


In [109]:
batch_size = 256
nb_epoch=20

In [110]:
#train_X1 and train_y1 are the augmented data alone to check accuracy only on augmented data 
#feed the model.fit only with these
train_X1 = train_X1.values
train_y1 = train_y1.values

In [111]:
#Checking Accuracy with training+augmented data train_X and train_y are 'train + augmented' data
history = model.fit(train_X1, train_y1, batch_size=batch_size, epochs=nb_epoch,verbose=2, 
                    validation_data=(X_val , Y_val),callbacks=[saveBestModel])

Train on 70000 samples, validate on 928 samples
Epoch 1/20
Epoch 00000: val_acc improved from -inf to 0.09159, saving model to /home/satishk/saved_weights/best_weights_2l_MLP_11.hdf5
10s - loss: 2.6435 - acc: 0.0705 - val_loss: 2.8042 - val_acc: 0.0916
Epoch 2/20
Epoch 00001: val_acc did not improve
9s - loss: 2.6392 - acc: 0.0705 - val_loss: 2.7999 - val_acc: 0.0819
Epoch 3/20
Epoch 00002: val_acc did not improve
9s - loss: 2.6392 - acc: 0.0696 - val_loss: 2.8034 - val_acc: 0.0819
Epoch 4/20


KeyboardInterrupt: 

In [ ]:
#Frame label accuracy
score = model.evaluate(X_val, Y_val, verbose=0)
print('ERROR after Data Augmentation %:', 1-score[1])

In [ ]:
S_encoder

In [ ]:
X_train_30sec_gen = Variable(torch.from_numpy(X_train_10sec_F))

In [ ]:
S_encoder.eval()

In [ ]:
gen_samples = S_encoder(X_train_30sec_gen.cuda())

In [ ]:
gen_samples.data.shape

In [ ]:
#Taking the generated iVectors we will try to check the acc by MLP

In [ ]:
train_X_gen = gen_samples.cpu().data.numpy()

In [ ]:
type(train_X_gen)

In [ ]:
X_train = pd.DataFrame(X_train_10sec)
#Y_train = pd.DataFrame(Y_train)


In [ ]:
val_lre = pd.read_csv('/home/satishk/GAN_lre/gan_csv/dev_feat_BNF_h5_05Jan_10sec.csv')


In [ ]:
val_lre.head()

In [ ]:
X_val = val_lre.drop(["language_code","uttid","segmentid1","data_source","speech_duration"],axis=1)
y_val = val_lre["language_code"]
y_val_segmentid = val_lre["segmentid1"]

In [ ]:
X_val = X_val.values
#X_eval = X_eval.values

In [ ]:
#X_eval.shape

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(y_val)

In [ ]:
le.classes_

In [ ]:
y_val_labels = le.transform(y_val)

In [ ]:
y_val_labels[0:25]

In [ ]:
# convert class vectors to binary class matrices
Y_val = np_utils.to_categorical(y_val_labels, nb_classes)
#Y_eval = np_utils.to_categorical(y_eval_labels, nb_classes)

In [ ]:
Y_val

In [ ]:
X_val = X_val.astype('float32')


In [ ]:
X_val_10sec_gen = Variable(torch.from_numpy(X_val))

In [ ]:
gen_samples_val = S_encoder(X_val_10sec_gen.cuda())

In [ ]:
gen_samples_val.data.shape

In [ ]:
X_val_gen = gen_samples_val.cpu().data.numpy()

In [ ]:
model = Sequential()
model.add(Dense(512, input_dim=500))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [ ]:
model.summary()

In [ ]:
adam=optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
best_weights_filepath = '/home/satishk/saved_weights/best_weights_2l_MLP_11.hdf5'
saveBestModel = ModelCheckpoint(best_weights_filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='auto')


In [ ]:
batch_size = 256
nb_epoch = 15

In [ ]:
#train_X_gen
#Checking Accuracy with augmented data 
history = model.fit(train_X_gen, Y_train_labels , batch_size=batch_size, epochs=nb_epoch,verbose=2, 
                    validation_data=(X_val_gen , Y_val),callbacks=[saveBestModel])

In [ ]:
score = model.evaluate(X_val_gen, Y_val, verbose=0)
print('Acc %:', score[1])

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC


In [ ]:
classif = SVC(C=1, kernel='rbf', gamma='auto', coef0=1, shrinking=True, random_state=0,
                      probability=False, tol=1e-3, cache_size=1e4, class_weight='balanced')
classif.fit(X_train_10sec_F, y_30sec_labels)

In [ ]:
classif.classes_

In [ ]:
acc = metrics.accuracy_score(y_val_labels, classif.predict(X_val))

In [ ]:
acc